In [1]:
from astropy.coordinates import EarthLocation,SkyCoord
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude  # Angles
from astropy.coordinates import AltAz
from astropy.time import Time, TimeDelta
from astropy import units as u
t=[Time('2023-10-01T11:00:00',format="isot",scale="utc")]
breakout=lambda la,lo,ht:{'lat':la,'lon':lo,'height':ht*u.m}
earthSpots={"Bunker Hill Monument":[42.37630000,-71.06070000,30.0,1531609,-4463479,4276597,844.2,932.4,19.9],
            "Old South Church":[42.35020000,-71.07800000,20.0,1530895,-4465783,4274447,-579.1,-1968.3,9.7],
            "Old North Church":[42.36640000,-71.05440000,20.0,1532341,-4464003,4275779,1366.7,-166.3,9.9],
            "USS Constitution":[42.37250000,-71.05660000,5.0,1532016,-4463622,4276267,1182.4,509.0,-5.1],
            "Museum of Science":[42.36790000,-71.07100000,10.0,1531010,-4464333,4275895,0.0,0.0,0.0],
            "Kendall Square":[42.36570000,-71.08230000,7.0,1530181,-4464795,4275706,-934.4,-252.5,-3.1],
            "Hatch Shell":[42.35790000,-71.07390000,4.0,1531021,-4465118,4275068,-244.4,-1113.7,-6.1],
            "Frog Pond":[42.35620000,-71.06580000,15.0,1531700,-4465028,4274936,427.8,-1302.1,4.9],
            "Public Garden":[42.35400000,-71.07000000,8.0,1531421,-4465292,4274751,77.8,-1545.9,-2.2],
            "Public Library":[42.34950000,-71.07820000,20.0,1530898,-4465841,4274385,-595.0,-2052.0,9.6],
            "MIT Dome":[42.35990000,-71.09200000,40.0,1529573,-4465489,4275251,-1733.8,-898.3,29.7]}
             
print (f"Location Baselines:{len(earthSpots)*(len(earthSpots)-1)/2}")
# Bunker Hill Monument: 42.37633621914482, -71.06072544082996
observing_location = EarthLocation(**breakout(*(earthSpots['Bunker Hill Monument'][0:3])))  
#observing_location = EarthLocation(lat='42.37633621914482', lon='-71.06072544082996', height=30*u.m)  
observing_time = t[0]  
aa = AltAz(location=observing_location, obstime=observing_time)


#Rigel and Betelgeuse
coords = ["05:14:33.700000	-8.200806", "05:55:11.700000	7.407667"]
c = SkyCoord(coords, unit=(u.hourangle, u.deg))
#print(c)
#coord = SkyCoord('4h42m', '-38d6m50.8s')
print(f"SkyCoord original: {c}")
print(f"Transformed to AA:{c.transform_to(aa)}")


Location Baselines:55.0
SkyCoord original: <SkyCoord (ICRS): (ra, dec) in deg
    [(78.64041667, -8.200806), (88.79875   ,  7.407667)]>
Transformed to AA:<SkyCoord (AltAz: obstime=2023-10-01T11:00:00.000, location=(1531612.29777372, -4463480.79182169, 4276594.14281663) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt) in deg
    [(210.44824942, 34.56128858), (204.60635023, 52.69407351)]>


In [18]:
breakout(*(earthSpots['Bunker Hill Monument']))

{'lat': 42.37633621914482,
 'lon': -71.06072544082996,
 'height': <Quantity 30. m>}

In [2]:
from astropy.time import TimeDelta


In [5]:
import numpy as np
t=[Time('2023-10-01T17:00:00',format="isot",scale="utc")]
dt = TimeDelta(1, format='jd')
t[0] + dt * np.linspace(0., 1., 13)

<Time object: scale='utc' format='isot' value=['2023-10-01T17:00:00.000' '2023-10-01T19:00:00.000'
 '2023-10-01T21:00:00.000' '2023-10-01T23:00:00.000'
 '2023-10-02T01:00:00.000' '2023-10-02T03:00:00.000'
 '2023-10-02T05:00:00.000' '2023-10-02T07:00:00.000'
 '2023-10-02T09:00:00.000' '2023-10-02T11:00:00.000'
 '2023-10-02T13:00:00.000' '2023-10-02T15:00:00.000'
 '2023-10-02T17:00:00.000']>